In [10]:
import pandas as pd
import os

C:\Users\Alan\Anaconda\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
#Checking to see whether the data is downloaded, else, it will download
if not os.path.isfile("../Data/tweetDBtoy.txt"):
    from DataDL import download_source as dld
    url = "https://www.dropbox.com/s/ysfgbbowvlo9zji/tweetDBtoy.txt?dl=1"
    pth=''
    print "Downloading twittertoyData\nThis may take a while..."
    files = dld(url, pth=pth, zp=False)
else:
    print "you seem to have the Twitter data. We can proceed"

you seem to have the Twitter data. We can proceed


In [11]:
toyds = pd.read_csv("../Data/tweetDBtoy.txt")
#locationdf = toyds['"tweet"']["location"]
tweet_list = list(toyds[' "tweet"'])
toyds.head()

C:\Users\Alan\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,"""timestamp""","""lon""","""lat""","""tweet""","""user_id""","""rtwts""","""fvrts""","""application""","""source"""
0,662012596569018368,1446689077,-74.075596,40.626242,"Flood Control Staten Island, NY $50 off Emerge...",128961312,NaN,NaN,NaN,twitter1.db
1,662012914115612672,1446689152,-74.210701,40.663992,We're #hiring! Read about our latest #job open...,28447324,NaN,NaN,NaN,twitter1.db
2,662013116390109184,1446689201,-74.274952,40.592877,Want to work at Macy's Retail? We're #hiring i...,28480310,NaN,NaN,NaN,twitter1.db
3,662013252549935104,1446689233,-74.021759,40.619148,Fall feels 🍁 #salvocapellisalon @ Salvo Capel...,550593958,NaN,NaN,NaN,twitter1.db
4,662014669104640000,1446689571,-74.106072,40.630363,Can you recommend anyone for this #job? Shift ...,22454315,NaN,NaN,NaN,twitter1.db


In [21]:
def location_extractor(tweet):
    symbol_list = [',','.','<','>','/','?',':',';','[',']','{','}','|','\\','=','+','_',')','(','*','^','%','$','#','@','!']
    #note - I delibrately omitted '-' and '&' from this list, because they often form part of the place names. but you can put them in again.
    preposition_list=['at', 'At', 'in', 'In', '@', 'to', 'To', '2', 'From', 'from', 'through', 'Through', 'near', 'Near']
    misleading_list=['want','Want','click', 'Click', 'its', "it's", 'Its', "Its"]
    tweetwords = tweet.split()

#     print "Before: {}".format(tweetwords)
    indx = 0
    for word in tweetwords:
        if word[:1] in symbol_list:
            symb = word[:1]
            indx+=1
            if word[1:]=="":#sometimes symbols are by themselves anyway. 
                continue #This prevents unnecessarily inserting an empty list item
            else:
                tweetwords.insert(indx,word[1:])
                indx-=1
                tweetwords[indx]=symb
        elif word[-1:] in symbol_list:
            symb = word[-1:]
            tweetwords[indx]=word[:-1]
            indx+=1
            tweetwords.insert(indx,symb)
            indx-=1
        indx+=1
#     print "\nAfter: {}".format(tweetwords)
    
    locations = []
    loc_word_flag = False
    misl_flag = False
    loc_initial_flag=False
    for word in tweetwords:
    #     print "\nIn For. Word:{}".format(word)#unnecessary
        if misl_flag == True:
            misl_flag = False
            continue
        elif loc_word_flag == False:
    #         print "in if 1 "#unnecessary
            if word in misleading_list:
                misl_flag = True
                continue
            elif word in preposition_list:
    #             print "in if 2 "#unnecessary
                loc_word_flag=True
                locations.append("")
                location_words=0
                continue
            else:
    #             print "in else 3 "#unnecessary
                continue
        else: #this section onwards is for words we think might be locations...
    #         print "in else 4 "#unnecessary
            if location_words==0 and word in symbol_list:
    #             print "in if 5 "#unnecessary
                continue # this enables us to catch locations that are tagged with a '#'
        
        #New code 12:40:
            elif loc_initial_flag==True and word in symbol_list:
                loc_initial_flag=False
                continue 


            elif word not in preposition_list and word not in symbol_list:
    #             print "in if 6 "#unnecessary
                if word[:4]=='http':
                    location_words=0
                    loc_word_flag=False
                    continue
                else:
                    loc_pos=len(locations)-1
                    locations[loc_pos]+=word+" "
                    location_words+=1
        #New code 12:40:
                    if len(word)==1:
                        loc_initial_flag=True
                    if location_words<4:
        #                 print "in if 7 "#unnecessary
                        continue
                    else:
        #                 print "in else 8 "#unnecessary
                        location_words=0
                        loc_word_flag=False
                        continue
            elif word in preposition_list:
    #             print "in else 9 "#unnecessary
                loc_word_flag=True
                locations.append("")
                location_words=0
                continue
            else:
    #             print "in else 10 "#unnecessary
                location_words=0
                loc_word_flag=False
                continue

    return locations

    

In [20]:
tst_size=500
for tweet in tweet_list:
    print tweet
    loc = location_extractor(tweet)
    print loc
    tst_size-=1
    if tst_size<0:
        break

Flood Control Staten Island, NY $50 off Emergency Servi https://t.co/mQjLAut1YA StatenIsland Water Extraction &amp; Damage Restoration #coupon
[]
We're #hiring! Read about our latest #job opening here: Maintenance &amp; Engineering Supervisor 48077 - https://t.co/XvnTmG7kjH #Elizabeth, NJ
[]
Want to work at Macy's Retail? We're #hiring in #Woodbridge, NJ! Click for details: https://t.co/AIyd8V24wG #Retail #Job #Jobs #CareerArc
["Macy's Retail ", 'Woodbridge ']
Fall feels 🍁 #salvocapellisalon @ Salvo Capelli Salon https://t.co/Cz31wFVqCZ
['Salvo Capelli Salon ']
Can you recommend anyone for this #job? Shift Supervisor (US) - https://t.co/xq1iL4m7U7 #Hospitality #Veterans #Hiring #CareerArc
[]
Stop by for your freshly baked Irish soda bread!!!!! 
Irish soda bread season came early to Jean… https://t.co/bWmn6vW1Mw
['Jean\xe2\x80\xa6 ']
But why? 🙄 @ Staten Island Ferry https://t.co/Py2SAG6bYO
['Staten Island Ferry ']
250 Ritek Ridata Dual Layer 8.5GB 4X DVD-R DL (Logo on Top) $255.00 #Free